In [ ]:
import sys
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from copy import deepcopy
from tqdm import tqdm
import random
from random import shuffle

In [ ]:
import cv2
import numpy as np
import random
import os
import re
from tqdm import tqdm
from copy import deepcopy
import sys

import cv2
import numpy as np
from tqdm import tqdm
from multiprocessing import Process, Manager


In [ ]:
from IPython.display import clear_output
from matplotlib import pyplot as plt
import collections
%matplotlib inline

def live_plot(data, figsize=(16,8), title=''):
    clear_output(wait=True)
    plt.figure(figsize=figsize)
    plt.imshow(convert_to_rgb(data.astype(int)))
    plt.title(title)
    plt.show()

In [ ]:
manager = Manager()  # create only 1 mgr
# prob_dict = manager.dict()

## 读取数据

In [ ]:
def convert_to_rgb(img):
    img = deepcopy(img)
    b,g,r = cv2.split(img)  
    img = cv2.merge([r,g,b])
    return img

In [ ]:
background = cv2.imread('../../base_data/background.jpeg')
hedge = cv2.imread('../../base_data/hedge.jpeg')

## 生成视频代码

In [ ]:
def generate_video(frame_dir=None, video_dir=None, video_name='video.avi'):

    image_num = len([i for i in os.listdir(frame_dir) if '.jpg' in i])
    image_name_list = [f'{i}.jpg' for i in range(image_num)]
    frame = cv2.imread(os.path.join(frame_dir, image_name_list[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(f'{video_dir}/{video_name}', 0, 20, (width, height))

    # video = cv2.VideoWriter(f'{self.video_dir}/{video_name}', 0, 20, (width, height))

    for image in image_name_list:
        video.write(cv2.imread(os.path.join(frame_dir, image)))

    cv2.destroyAllWindows()
    video.release()

## 定义参数

In [ ]:
speed_dict = {
    "background": 1,
    "hedge_1": 2,
    "hedge_2": 3,
    "black_bar": 4
}

crop_height = 900
crop_width = 1500
slide_width = 800
camera_width = 250

crop_width_extend = crop_width + 1000

size_1 = camera_width * 2
size_2 = slide_width

dimension = 3
black_bar_width = 5

background_prob = 0.9
noise_average_size = 10

vertical_movement = [0]
horizontal_movement = [1, 2]

In [ ]:
background_crop = deepcopy(background[-crop_height:, :crop_width, :])
# live_plot(background_crop)

In [ ]:
hedge_crop = deepcopy(hedge[-crop_height:, :crop_width, :])
# live_plot(hedge_crop)

In [ ]:
def rgb_mean(image):
    return tuple([np.mean([np.mean(image[:, :, i])]) for i in range(2, -1, -1)])

def initialise_image(image):
    reshape_size = image.shape[0] * image.shape[1]
    a = np.zeros((dimension, reshape_size))
    for i in range(3):
        a[i] = np.array(image[:, :, i]).reshape(1, reshape_size)
    return a

def random_select():
    return random.randint(0, 250), random.randint(0, 750)

def random_radius():
    return random.randint(10, 35)

def mahalanobis(x, mean, cov):
    return np.sqrt(np.linalg.inv(cov).dot(np.transpose(x - mean)).dot((x - mean)))

def cal_ratio_prob(x):
    return 1 - (x/x.sum())

def ratio_soft_max(self, x):
    return ratio_prob(soft_max(x))

def soft_max(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [ ]:
hedge_mean = np.array(rgb_mean(hedge_crop))
background_mean = np.array(rgb_mean(background_crop))
hedge_cov = np.cov(initialise_image(hedge_crop))
background_cov = np.cov(initialise_image(background_crop))

## 显示图片

In [ ]:
# plt.imshow(convert_to_rgb(background))
# plt.show()
# plt.imshow(convert_to_rgb(hedge))
# plt.show()

In [ ]:
background.shape, hedge.shape

## 定义参数

In [ ]:
background_crop = deepcopy(background[-crop_height:, :crop_width, :])
# live_plot(background_crop)

In [ ]:
hedge_crop = deepcopy(hedge[-crop_height:, :crop_width, :])
# live_plot(hedge_crop)

In [ ]:
def generate_black_bar_image():
    black_bar_image = np.zeros((crop_height, crop_width_extend, dimension))
    for i in range(150, crop_width_extend, 200):
        black_bar_image[:, i - black_bar_width:i + black_bar_width, :] = 1
    return black_bar_image

def generate_random_hole():
    random_hole = np.random.random_sample((crop_height + noise_average_size,
                                           crop_width_extend + noise_average_size))
    random_hole = np.round(random_hole - (background_prob / 2))
    group_random_hole = np.zeros((crop_height, crop_width))
    for i in range(group_random_hole.shape[0]):
        for j in range(group_random_hole.shape[1]):
            group_random_hole[i, j] = random_hole[i:i + noise_average_size, j:j + noise_average_size].mean()
    group_random_hole = np.round(group_random_hole / group_random_hole.max())
    group_random_hole = np.array([[[j] * 3 for j in i] for i in group_random_hole])
    return group_random_hole.astype(int)

In [ ]:
def calculate_frame_prob(frame):
    frame_prob = np.zeros(frame.shape)
    for i in range(frame.shape[0]):
        for j in range(frame.shape[1]):
            pixel = frame[i][j]
            hedge_distance = mahalanobis(pixel, hedge_mean, hedge_cov)
            top_distance = mahalanobis(pixel, background_mean, background_cov)
            ratio_prob = cal_ratio_prob(np.array([top_distance, hedge_distance]))[0]
            frame_prob[i][j] = round(ratio_prob)
    return frame_prob

In [ ]:
random_hole_1 = generate_random_hole()
# live_plot(random_hole_1 * 255)

random_hole_2 = generate_random_hole()
# live_plot(random_hole_2 * 255)

black_bar_image = generate_black_bar_image()
# live_plot(((1 - black_bar_image)*255).astype(int))

# random_hole_1 = generate_random_hole()
# random_hole_2 = generate_random_hole()
# black_bar_image = generate_black_bar_image()
verti_pos = int(background_crop.shape[0]/2)
hori_pos = 0

image_dict = {}

for i in tqdm(range(10000), ncols=70):
    try:
        upper, lower = verti_pos + camera_width, verti_pos - camera_width
        background_pos = int(speed_dict['background'] * hori_pos)
        hedge_1_pos = int(speed_dict['hedge_1'] * hori_pos)
        hedge_2_pos = int(speed_dict['hedge_2'] * hori_pos)
        black_bar_pos = int(speed_dict['black_bar'] * hori_pos)

        background_crop_temp = background_crop[int(lower):int(upper), background_pos:background_pos + slide_width, :]
        hedge_1_image_temp = hedge_crop[int(lower):int(upper), hedge_1_pos:hedge_1_pos + slide_width, :]
        random_hole_1_temp = random_hole_1[int(lower):int(upper), hedge_1_pos:hedge_1_pos + slide_width, :]

        hedge_2_image_temp = hedge_crop[int(lower):int(upper), hedge_2_pos:hedge_2_pos + slide_width, :]
        random_hole_2_temp = random_hole_2[int(lower):int(upper), hedge_2_pos:hedge_2_pos + slide_width, :]

        black_bar_image_temp = black_bar_image[int(lower):int(upper), black_bar_pos:black_bar_pos + slide_width, :]

        output_image = background_crop_temp * (1 - random_hole_1_temp) + hedge_1_image_temp * random_hole_1_temp
        output_image = output_image * (1 - random_hole_2_temp) + hedge_2_image_temp * random_hole_2_temp
        output_image = output_image * (1 - black_bar_image_temp) + black_bar_image_temp
        output_image = output_image.astype(int)
#         cv2.imwrite(f"../scene_data/{i}_{verti_pos}_{hori_pos}.jpg", output_image)
        image_dict[f"{i}_{verti_pos}_{hori_pos}.jpg"] = output_image.copy()
        hori_pos += random.choice(horizontal_movement)
        verti_pos += random.choice(vertical_movement)
    except Exception as e:
        print(e)
        break

In [ ]:
n = 4
m = 4
set_shape = (2*n, 2*n, 3)

In [ ]:
frame_dir = '../scene_data'

scene_dict = {}
for file_name in image_dict.keys():
    if file_name[0] != '.':
        num, verti_pos, hori_pos = file_name.replace('.jpg', '').split('_')
        scene_dict[int(num)] = {'file_name': file_name, 'num':num, 'verti_pos':int(verti_pos), 'hori_pos':int(hori_pos)}


In [ ]:
# scene_dict

In [ ]:
result_compare_list = []

for _ in tqdm(range(100), ncols=70):
    compare_list = []
    scene_num = random.choice(list(scene_dict.keys()))
    for num in [scene_num, scene_num + 1]:
        file_name = scene_dict[num]['file_name']
        verti_pos = scene_dict[num]['verti_pos']
        hori_pos = scene_dict[num]['hori_pos']
        frame = np.array(image_dict[file_name].copy())
        compare_list.append([frame, (verti_pos, hori_pos), num])
    ground_truth = np.array(compare_list[1][1]) - np.array(compare_list[0][1])

    compare_dict = {}
    frame = compare_list[0][0]
    compare_frame = compare_list[1][0]
    for _ in range(32):
        while 1:
            frame_i, frame_j = random.choice(range(frame.shape[0])), random.choice(range(frame.shape[1]))
            pixels = frame[frame_i - n:frame_i + n, frame_j - n:frame_j + n, :]
            pixels_prob = calculate_frame_prob(pixels)
            if pixels.shape == set_shape and np.sum(pixels_prob) >= set_shape[0] * set_shape[1] * set_shape[2] * 0.8:
                break

        for i in range(frame_i-m, frame_i+m):
            for j in range(frame_j-m, frame_j+m):
                compare_pixels = compare_frame[i - n:i + n,j - n:j + n,:]
                if pixels.shape == set_shape and compare_pixels.shape == set_shape:
    #                 pixels_prob = calculate_frame_prob(pixels)
    #                 compare_pixels_prob = calculate_frame_prob(compare_pixels)
    #                 total_prob = pixels_prob * compare_pixels_prob

                    diff_array = np.array([i for i in ((pixels - compare_pixels)).reshape(-1,1) if i != 0])

                    diff = np.sum(np.power(diff_array,2))/len(diff_array)
                    compare_dict[(frame_i - i, frame_j - j)] = compare_dict.get((frame_i - i, frame_j - j), []) + [diff]
    compare_dict = {k:np.mean(v) for k,v in compare_dict.items()}
    compare_dict = {k: v for k, v in sorted(compare_dict.items(), key=lambda item: item[1])}
    predict = np.array(list(compare_dict.keys())[0])
    result_compare_list.append(list(ground_truth) == list(predict))
    print((ground_truth, predict))
print('{:.2f}%'.format(sum(result_compare_list)*100/len(result_compare_list)))
    